# Year 2023 Day 11


In [ ]:
import numpy as np
import pandas as pd
from pyobsplot import Plot # , js # can be used to pass js expr as str 
import xarray as xr


Note: the following logic can be reused for 202324 (show the vectors aligning progressively)

In [ ]:
from advent_of_code.y_2023.problem_202311 import AdventOfCodeProblem202311

problem = AdventOfCodeProblem202311()
problem

In [ ]:
puzzle_input = problem.parse_input_text_file()
puzzle_input

In [ ]:
grid = (puzzle_input == ord("#")).astype(int) * 255
grid

In [ ]:
from advent_of_code.y_2023.problem_202311 import get_compartiments




In [ ]:
from advent_of_code.y_2023.problem_202311 import (
    compute_adjacency_matrix,
    create_coord_array,
)


def visualize_puzzle_input(
    space_xda: xr.DataArray,
    with_rules: bool = False,
    with_graph: bool = False,
    coord_array: xr.DataArray | None = None,
    edge_density: float | None = None,
):
    marks = []
    marks.append(
        Plot.axisX({"anchor": "top"}),
    )
    marks.append(
        Plot.raster(
            grid.values.reshape(-1),
            {
                "width": grid.col.size,
                "height": grid.row.size,
                "imageRendering": "pixelated",
            },
        ),
    )

    if with_graph:
        if coord_array is None:
            raise ValueError(
                "coord_array was not given, it is needed to visualize the graph"
            )
        if edge_density is None:
            edge_density = 1

        coord_array = coord_array + 0.5

        lines = []
        for idx, pair_source in enumerate(coord_array.values.tolist()):
            for pair_target in coord_array.values[idx + 1 :].tolist():
                lines.append([pair_source, pair_target])

        # Only display a subset of all the links
        period = int(1 / edge_density)
        lines = lines[::period]

        table = {
            "x1": [],
            "y1": [],
            "x2": [],
            "y2": [],
        }

        for line in lines:
            table["y1"].append(line[0][0])
            table["x1"].append(line[0][1])
            table["y2"].append(line[1][0])
            table["x2"].append(line[1][1])

        table = pd.DataFrame.from_dict(table)
        marks.append(
            Plot.link(
                table,
                {
                    "x1": "x1",
                    "y1": "y1",
                    "x2": "x2",
                    "y2": "y2",
                    "stroke": "white",
                    "strokeWidth": 0.3,
                },
            )
        )

    if with_rules:
        row_chunks = get_compartiments(space_xda, "row", "col")
        col_chunks = get_compartiments(space_xda, "col", "row")

        marks.append(
            Plot.ruleY(
                [chunk.start - 0.5 for chunk in list(row_chunks.values())[1:]],
                {"stroke": "red"},
            ),
        )
        marks.append(
            Plot.ruleX(
                [chunk.start - 0.5 for chunk in list(col_chunks.values())[1:]],
                {"stroke": "red"},
            ),
        )

    return Plot.plot(
        {
            "height": 140 * 4,
            "width": 140 * 4,
            "color": {"scheme": "magma"},
            "x": {"domain": [0, grid.col.size], "label": "column"},
            "y": {"domain": [grid.row.size, 0], "label": "row"},
            "marks": marks,
        }
    )

In [ ]:
visualize_puzzle_input(puzzle_input)

In [ ]:
visualize_puzzle_input(puzzle_input, with_rules=True)

Let's count how many nodes there are in this graph:

In [ ]:
coord_array = create_coord_array(puzzle_input)
node_count = coord_array["z"].size
node_count

Now we can calculate the number of edges in the related [Complete graph](https://en.wikipedia.org/wiki/Complete_graph), where every pair of node is connected:

In [ ]:
n = node_count
edge_count = n*(n-1)//2 # 2 among n
edge_count

The amount is very high, hence in the visualisation, only a fraction is shown

In [ ]:
visualize_puzzle_input(
    puzzle_input, with_rules=True, with_graph=True, coord_array=coord_array, edge_density=0.005
)

In [ ]:
visualize_puzzle_input(
    puzzle_input,
    with_rules=True,
    with_graph=True,
    coord_array=coord_array,
    edge_density=0.03,
)

The key to expansion is to assign to every chunk a node (it creates a [Lattice graph](https://en.wikipedia.org/wiki/Lattice_graph)).
Then the same logic can be applied on chunks instead of stars, and the distance
becomes the sum of both adjacency matrix, with the chunk one multiplied by
the expansion coefficient